<a href="https://colab.research.google.com/github/joiedevivre02/joiedevivre02/blob/main/superapp_scraping_gplay_autoupdate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up our Collab!

In [ ]:
new_date = "2023-05-31"

In [ ]:
# Mounting on your google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Finding the folder location
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# Checking if we're in the right folder by checking the contents
#%ls

In [ ]:
# Install the package we'll be using!
# In Google Colab, pandas and numpy are downloaded by default :D
!pip install google-play-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing Packages
from google_play_scraper import app
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews_all

## Scraping using the package

In [ ]:
# Using the google_play_scraper's function to scrape the data from the respective app
# https://pypi.org/project/google-play-scraper/#description
# https://github.com/JoMingyu/google-play-scraper


konsulta_reviews = reviews_all(
    'com.globaltelehealth.konsultamd',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)





In [ ]:
# we turn the konsulta reviews data into a pandas dataframe so we can analyze and edit it!
# we use from_records to turn it from structured data into a df via https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html
# .head() shows a sample of the data, the top 5. .tail() shows the bottom 5

df_busu = pd.DataFrame.from_records(konsulta_reviews)
df_busu.tail()
df_busu['reviewId'].nunique()

2080

## Formatting our Dataframe
To make it look like what you want it to look like!

In [ ]:
# We're dropping the columns you don't need since we only need Username, Rate, Date, Review
# axis 0 are rows while axis 1 are columns so we're dropping columns
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df_busu.drop(['reviewId', 'userImage','thumbsUpCount', 'reviewCreatedVersion', 'replyContent', 'repliedAt'], axis = 1, inplace=True)

# now we're left with the 4 columns you want, we need to format them and move em around
df_busu.head()

,userName,content,score,at,appVersion
0,Felesicimo Lagaret,Very bad please stop Pop to my device this app...,1,2023-06-20 03:43:35,5.0.68
1,Ryan Estavas,A helpful companion for your health and wellbe...,3,2023-06-20 02:17:15,5.0.68
2,Mhark Henrie Dela Cruz,I selected the chat option but was being route...,1,2023-06-19 10:08:32,5.0.68
3,Irish apple Caylao,Scammer,1,2023-06-19 07:44:50,None
4,Al Francis Silos (unHurt),Already paid for the consultation. Then redire...,1,2023-06-19 07:18:56,5.0.68


In [ ]:
# this renames the columns so "old_name":"new_name"
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
df_busu.rename(columns={"userName": "UserName", "content": "Review", "score": "Rating", "at": "Date", }, inplace=True)

# We turn the datetime into a date only!
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.html
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html
# since "Date" is already a datetime, we use dt.date to return only the date w/o the time. BUT it becomes a string
# we then use astype to convert the string back to a datetime object
df_busu['Date'] = df_busu['Date'].dt.date.astype('datetime64')

# Swapping the column positions narin with .reindex()
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html
df_busu = df_busu.reindex(columns=['Date', 'UserName', 'Rating','Review'])
df_busu.tail()

,Date,UserName,Rating,Review
2075,2020-05-23,Alvin Daquiwag,5,Verry nice .apps..sana lahat ng apps. tulad ni...
2076,2020-05-22,Alvince Kley Magos,5,Good
2077,2020-05-21,CRISOLITO ABAD,5,Good
2078,2020-05-15,G Syq,5,Great!!! Easy and convenient 👍
2079,2020-05-15,Patrick Randell Mojica,5,Good project. Nice doctors.


In [ ]:
#change date here
df_busu = df_busu.loc[df_busu["Date"] > new_date]
df_busu.sort_values(by = ["Date"], ascending=True, inplace=True)
df_busu.reset_index(drop=True, inplace=True)
df_busu.head()

<ipython-input-10-798007ede1aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_busu.sort_values(by = ["Date"], ascending=True, inplace=True)


,Date,UserName,Rating,Review
0,2023-06-01,Robert Po,1,I tried logging in after the migration of my a...
1,2023-06-01,Dannah Socudig,4,Ommvl'e'. N mn. ' '''n ' mz2 2:
2,2023-06-01,Alexer Flores,5,Superb
3,2023-06-02,Daniel Espino,4,Great app. It just takes time for me to receiv...
4,2023-06-03,Daniel Dougie,4,"Öa,"


In [ ]:
# Since it looks good na, we'll save it as a CSV and it'll be saved in the folder you're currently in!
# I added "data/" to save it in the data folder, which is a folder inside my current folder
# index = False means we won't keep the numbering / index in the CSV when we save it

df_busu.rename(columns = {'Date': 'DATE',
                          'UserName': 'REVIEWER NAME / LAZADA ACCOUNT NAME',
                          'Rating': 'STARS',
                          'Review': 'SPECIFIC PRODUCT REVIEW WRITE-UP'
                          }, inplace = True)

#df_busu.to_csv('konsulta_reviews_gplay.csv', index=False)
import datetime
df_busu['DATE']= pd.DatetimeIndex(df_busu['DATE'])
df_busu['DATE']= pd.to_datetime(df_busu['DATE'])

df_busu['MONTH'] = pd.DatetimeIndex(df_busu['DATE']).month

import pandas as pd

pd.Series.dt.week_start = 0
df_busu['WEEK'] = df_busu['DATE'].dt.week


<ipython-input-11-e1f626483351>:21: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_busu['WEEK'] = df_busu['DATE'].dt.week


In [ ]:
#authenticate
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#import
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F5V-a9f8UrQd3fnn61NcXmDFlgttASSZKXlPPbhnQa8/edit#gid=588261511')



#@title Import google sheets and select first row as header

googleplay = wb.worksheet('Google Play Reviews')
googleplay = googleplay.get_all_values()
googleplay = pd.DataFrame(googleplay)
googleplay.columns = googleplay.iloc[0].str.upper()
googleplay = googleplay.iloc[1:]


In [ ]:
#Step 5 - Append the dataframes
df = googleplay.append(df_busu)
df

<ipython-input-13-a1a06b73d74b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = googleplay.append(df_busu)


,MONTH,WEEK,DATE,REVIEWER NAME / LAZADA ACCOUNT NAME,STARS,SPECIFIC PRODUCT REVIEW WRITE-UP,KEY ISSUE (MANUALLY INPUT),"IF VALUE IN L > 0, THEN ""INTERN REVIEW""",CONCATENATE,ISSUE CATEGORY,ISSUE OWNER,INTERN?
1,1,4,2023-01-28 0:00:00,Mr. Drew TV,5,PERFECT APPS 💙💙💙 THANK YOU KMD,App related positive review,,,,,
2,2,6,2023-02-06 0:00:00,Suchi,5,This a very user-friendly version of KonsultaM...,Doctor/Service related positive review,,,,,
3,2,6,2023-02-07 0:00:00,Jean Romestain,1,Useless as it doesn't find my existing subscri...,General negative review,,,,,
4,2,6,2023-02-07 0:00:00,Aaron Zuñiga,5,"Very smooth transaction, well trained doctors ...",Doctor/Service related positive review,,,,,
5,2,7,2023-02-15 0:00:00,Ezia Lacuata,1,"Availed of chat consultation, no one showed up...",General negative review,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
39,6,25,2023-06-19 00:00:00,Al Francis Silos (unHurt),1,Already paid for the consultation. Then redire...,NaN,NaN,NaN,NaN,NaN,NaN
40,6,25,2023-06-19 00:00:00,Irish apple Caylao,1,Scammer,NaN,NaN,NaN,NaN,NaN,NaN
41,6,25,2023-06-19 00:00:00,Mhark Henrie Dela Cruz,1,I selected the chat option but was being route...,NaN,NaN,NaN,NaN,NaN,NaN
42,6,25,2023-06-20 00:00:00,Ryan Estavas,3,A helpful companion for your health and wellbe...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Step 6 - Open the Google Sheet file you want to write your new data in and then insert the data with the set function
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#wb3 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F5V-a9f8UrQd3fnn61NcXmDFlgttASSZKXlPPbhnQa8/edit#gid=0')
wb3 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1HlcGbSZThdw0gGzsT9mVpDOQrOj9k8Pzxq0r8Bzi3Wk/edit#gid=0')



sheet3 = wb3.worksheet('Google Play Reviews')
set_with_dataframe(sheet3, df)